In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Data
Here is where you can download the data from EPA: https://www.epa.gov/outdoor-air-quality-data/download-daily-data

Select: 
- Pollutant = PM2.5
- Year = 2020 thru 2024 (individual downloads for each, sigh)
- Geographic Area = Houston - The Woodlands - Sugar Land CBSA + All Sites

You can see from the codeblock below that subsequent downloads of past data aren't always identical, telling me that they likely continue to update values long after the dates have passed. So just download files fresh every time you're doing this. I would assume the further back you go, the less likely things are to change but... again... I'm not wasting time on inspecting that now.

## Process
Ok so we're looking for monitors that are over 9 and 12 (new and old EPA limits). You take the annual average concentration for each monitor, then you take the average of those annual averages over 3-year periods. If the 3-year avg is over the concentration limit, then we flag the place. Here's [an EPA source](https://experience.arcgis.com/experience/a2ca272ce9fc4019a88ce35b863e2cab#data_s=id%3AdataSource_1-18a8e80642b-layer-19-19147d8da1c-layer-32%3A951) we can use to compare 3-year values for each monitor for fact-checking purposes.

- Remove days known to have high pollution not from industry: NYE, 4th of July
- Group by `Site ID` and `Daily Mean PM2.5 Concentration` and `year` to get annual average concentration for each site, for each year 
- Further group by `Site ID` and `Daily Mean PM2.5 Concentration` and `avgAnnualConc` to calculate 3-year average for timespans 2018-20, 2019-21, 2020-22, 2021-23, 2022-24
- On this grouped data, calculate how many of the past 3-year groupings the site has been higher than current 9 limit
- On this grouped data, calculate how many of the past 3-year groupings the site has been higher than past 12 limit
- On your daily data, calculate how many days per year each site had daily averages higher than 35
- Join grouped data to daily overage data

In [2]:
#just checking to make sure the 2023 data didn't change between now and the
#last time I downloaded it.
base_path = '../data/EPA/'
file_suffix = '_houmetro_epa_ad_viz_plotval_data'

df2023 = pd.read_csv(f'{base_path}2023{file_suffix}.csv')
df2023v2 = pd.read_csv(f'{base_path}2023v2{file_suffix}.csv')

print(len(df2023))
print(len(df2023v2))

#combine then dedup to see if they are the same
df2023combo = pd.concat([df2023, df2023v2])

print(len(df2023combo))

df2023combo.drop_duplicates(inplace=True)

#ok i don't actually care right now what the differences are. I just need to know
#that there are differences so i'm going to just download all the files i need
#fresh everytime i run this script.
print(len(df2023combo))

4964
4964
9928
4978


In [3]:
base_path = '../data/EPA/'
file_suffix = '_houmetro_epa_ad_viz_plotval_data'

years = range(2018, 2025)
pm25_dfs = []
for year in years:
    df = pd.read_csv(f'{base_path}{year}{file_suffix}.csv')
    df['datetime'] = pd.to_datetime(df['Date'])
    df['year'] = df['datetime'].dt.year
    pm25_dfs.append(df)
    
pm25 = pd.concat(pm25_dfs)

In [4]:
print(pm25.dtypes)
print(pm25.datetime.min())
print(pm25.datetime.max())
display(pm25.head())

Date                                      object
Source                                    object
Site ID                                    int64
POC                                        int64
Daily Mean PM2.5 Concentration           float64
Units                                     object
Daily AQI Value                            int64
Local Site Name                           object
Daily Obs Count                            int64
Percent Complete                         float64
AQS Parameter Code                         int64
AQS Parameter Description                 object
Method Code                              float64
Method Description                        object
CBSA Code                                  int64
CBSA Name                                 object
State FIPS Code                            int64
State                                     object
County FIPS Code                           int64
County                                    object
Site Latitude       

,Date,Source,Site ID,POC,Daily Mean PM2.5 Concentration,Units,Daily AQI Value,Local Site Name,Daily Obs Count,Percent Complete,AQS Parameter Code,AQS Parameter Description,Method Code,Method Description,CBSA Code,CBSA Name,State FIPS Code,State,County FIPS Code,County,Site Latitude,Site Longitude,datetime,year
0,01/02/2018,AQS,481671034,1,4.7,ug/m3 LC,26,Galveston 99th Street,1,100.0,88101,PM2.5 - Local Conditions,145.0,R & P Model 2025 PM-2.5 Sequential Air Sampler...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,167,Galveston,29.254474,-94.861289,2018-01-02,2018
1,01/08/2018,AQS,481671034,1,7.1,ug/m3 LC,39,Galveston 99th Street,1,100.0,88101,PM2.5 - Local Conditions,145.0,R & P Model 2025 PM-2.5 Sequential Air Sampler...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,167,Galveston,29.254474,-94.861289,2018-01-08,2018
2,01/14/2018,AQS,481671034,1,3.4,ug/m3 LC,19,Galveston 99th Street,1,100.0,88101,PM2.5 - Local Conditions,145.0,R & P Model 2025 PM-2.5 Sequential Air Sampler...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,167,Galveston,29.254474,-94.861289,2018-01-14,2018
3,01/20/2018,AQS,481671034,1,4.6,ug/m3 LC,26,Galveston 99th Street,1,100.0,88101,PM2.5 - Local Conditions,145.0,R & P Model 2025 PM-2.5 Sequential Air Sampler...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,167,Galveston,29.254474,-94.861289,2018-01-20,2018
4,01/26/2018,AQS,481671034,1,6.4,ug/m3 LC,36,Galveston 99th Street,1,100.0,88101,PM2.5 - Local Conditions,145.0,R & P Model 2025 PM-2.5 Sequential Air Sampler...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,167,Galveston,29.254474,-94.861289,2018-01-26,2018


- Remove days known to have high pollution not from industry: NYE, 4th of July
- Group by `Site ID` and `Daily Mean PM2.5 Concentration` and `year` to get annual average concentration for each site, for each year 
- Further group by `Site ID` and `Daily Mean PM2.5 Concentration` to calculate 3-year average for timespans 2018-20, 2019-21, 2020-22, 2021-23, 2022-24
- On this grouped data, calculate how many of the past 3-year groupings the site has been higher than current 9 limit
- On this grouped data, calculate how many of the past 3-year groupings the site has been higher than past 12 limit
- On your daily data, calculate how many days per year each site had daily averages higher than 35
- Join grouped data to daily overage data

In [5]:
#Remove days known to have high pollution not from industry: NYE, 4th of July
holidays = ['2018-01-01','2018-07-04','2018-12-31',
            '2019-01-01','2019-07-04','2019-12-31',
            '2020-01-01','2020-07-04','2020-12-31',
            '2021-01-01','2021-07-04','2021-12-31',
            '2022-01-01','2022-07-04','2022-12-31',
            '2023-01-01','2023-07-04','2023-12-31',
            '2024-01-01','2024-07-04','2024-12-31']

pm25 = pm25.loc[~pm25['datetime'].isin(holidays)]

#Group by `Site ID` and `year` to get annual average `Daily Mean PM2.5 Concentration` for each site, for each year
by_site = pm25.groupby(['Site ID', 'year'])['Daily Mean PM2.5 Concentration'].mean().reset_index()

#Further group by `Site ID` and `Daily Mean PM2.5 Concentration` to calculate 3-year average for timespans 
# 2018-20, 2019-21, 2020-22, 2021-23, 2022-24
by_yr = pd.pivot_table(by_site, index='Site ID', columns='year', 
                       values='Daily Mean PM2.5 Concentration', aggfunc='mean').reset_index()
by_yr['avg2018_20'] = by_yr[[2018, 2019, 2020]].mean(axis=1,skipna=False)
by_yr['avg2019_21'] = by_yr[[2019, 2020, 2021]].mean(axis=1,skipna=False)
by_yr['avg2020_22'] = by_yr[[2020, 2021, 2022]].mean(axis=1,skipna=False)
by_yr['avg2021_23'] = by_yr[[2021, 2022, 2023]].mean(axis=1,skipna=False)
by_yr['avg2022_24'] = by_yr[[2022, 2023, 2024]].mean(axis=1,skipna=False)

#calculate how many of the past years the site has been higher than current and past limits
def over_limit(x, limit):
    return (x > limit).sum()

over_cols = list(range(2018,2025))
by_yr['yrs_over12'] = by_yr[over_cols].apply(lambda x: over_limit(x,12), axis=1)
by_yr['yrs_over9'] = by_yr[over_cols].apply(lambda x: over_limit(x,9), axis=1)

#calculate how many days per year each site had daily averages higher than 35
daily_over35 = pm25.groupby('Site ID').agg(days_over35=('Daily Mean PM2.5 Concentration', lambda x: over_limit(x, 35)),
                                           max_value=('Daily Mean PM2.5 Concentration', 'max'),
                                           min_value=('Daily Mean PM2.5 Concentration', 'min')
                                        ).reset_index()

pm25_site_summary = by_yr.merge(daily_over35, on='Site ID', how='left')

#lets also join with the sensor features we have
site_info = pm25[['Site ID','County FIPS Code', 'County', 'Site Latitude', 'Site Longitude']].drop_duplicates()
pm25_site_summary = pm25_site_summary.merge(site_info, on='Site ID', how='left')

#renaming some columns because i'm anal
rename_cols = {'Site ID':'site_id','County FIPS Code':'cnty_fips','County':'county',
               'Site Latitude':'latitude','Site Longitude':'longitude'}
pm25_site_summary.rename(columns=rename_cols,inplace=True)

#export for graphics and whatnot
pm25_site_summary['monitor_type'] = 'EPA'
pm25_site_summary.to_csv('../data/analyzed/houmetro-epa-pm25-site-summary.csv',index=False)

/var/folders/2g/mqj0d6v95vbfcqgf7kpgrsrc0000gr/T/ipykernel_19702/2276814622.py:10: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  pm25 = pm25.loc[~pm25['datetime'].isin(holidays)]


In [6]:
print(pm25_site_summary.columns)
display(pm25_site_summary)

Index([     'site_id',           2018,           2019,           2020,
                 2021,           2022,           2023,           2024,
         'avg2018_19',   'avg2019_21',   'avg2020_22',   'avg2021_23',
         'avg2022_24',   'yrs_over12',    'yrs_over9',  'days_over35',
          'max_value',    'min_value',    'cnty_fips',       'county',
           'latitude',    'longitude', 'monitor_type'],
      dtype='object')


,site_id,2018,2019,2020,2021,2022,2023,2024,avg2018_19,avg2019_21,avg2020_22,avg2021_23,avg2022_24,yrs_over12,yrs_over9,days_over35,max_value,min_value,cnty_fips,county,latitude,longitude,monitor_type
0,480391012,NaN,NaN,NaN,NaN,NaN,7.512500,7.746667,NaN,NaN,NaN,NaN,NaN,0,0,0,21.0,0.8,39,Brazoria,28.964394,-95.354974,EPA
1,481671034,4.617549,7.519178,8.345625,7.190643,7.995087,9.771338,7.080380,6.827451,7.685149,7.843785,8.319023,8.282268,0,1,3,50.9,0.1,167,Galveston,29.254474,-94.861289,EPA
2,482010024,9.083733,9.241451,9.830769,9.937958,10.186234,10.380679,10.792733,9.385318,9.670059,9.984987,10.168290,10.453215,0,7,4,42.9,0.0,201,Harris,29.901036,-95.326137,EPA
3,482010046,NaN,NaN,NaN,12.652193,11.772159,13.032551,12.473571,NaN,NaN,NaN,12.485634,12.426094,3,4,3,44.7,-0.8,201,Harris,29.828086,-95.284096,EPA
4,482010055,NaN,NaN,NaN,NaN,9.458482,10.837709,10.641994,NaN,NaN,NaN,NaN,10.312729,0,3,2,40.7,2.7,201,Harris,29.695729,-95.499219,EPA
5,482010058,9.151320,8.656410,9.997429,9.632670,10.496078,11.510029,10.460377,9.268386,9.428836,10.042059,10.546259,10.822161,0,6,4,50.2,-0.1,201,Harris,29.770698,-95.031232,EPA
6,482010066,NaN,NaN,NaN,8.220229,8.035484,9.416082,9.116975,NaN,NaN,NaN,8.557265,8.856180,0,2,3,43.6,-0.3,201,Harris,29.723333,-95.635833,EPA
7,482011034,10.962757,9.626496,9.945665,10.528783,9.786167,10.613636,10.076324,10.178306,10.033648,10.086872,10.309529,10.158709,0,7,6,47.4,0.0,201,Harris,29.767997,-95.220582,EPA
8,482011035,10.067232,10.083812,10.171237,10.654712,10.622436,11.320558,11.176190,10.107427,10.303254,10.482795,10.865902,11.039728,0,7,13,46.7,1.6,201,Harris,29.733726,-95.257593,EPA
9,482011039,8.247229,7.469241,8.771526,8.818121,8.709417,9.787113,10.036667,8.162666,8.352963,8.766355,9.104884,9.511066,0,2,8,47.7,-0.8,201,Harris,29.670025,-95.128508,EPA


In [7]:
#482010046 is Settegast
pm25.loc[pm25['Site ID'] == 482010046].sort_values('datetime')

,Date,Source,Site ID,POC,Daily Mean PM2.5 Concentration,Units,Daily AQI Value,Local Site Name,Daily Obs Count,Percent Complete,AQS Parameter Code,AQS Parameter Description,Method Code,Method Description,CBSA Code,CBSA Name,State FIPS Code,State,County FIPS Code,County,Site Latitude,Site Longitude,datetime,year
730,05/05/2021,AQS,482010046,1,8.3,ug/m3 LC,46,Houston North Wayside,1,100.0,88101,PM2.5 - Local Conditions,209.0,Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,201,Harris,29.828086,-95.284096,2021-05-05,2021
731,05/06/2021,AQS,482010046,1,13.6,ug/m3 LC,59,Houston North Wayside,1,100.0,88101,PM2.5 - Local Conditions,209.0,Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,201,Harris,29.828086,-95.284096,2021-05-06,2021
732,05/07/2021,AQS,482010046,1,16.1,ug/m3 LC,64,Houston North Wayside,1,100.0,88101,PM2.5 - Local Conditions,209.0,Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,201,Harris,29.828086,-95.284096,2021-05-07,2021
733,05/08/2021,AQS,482010046,1,8.8,ug/m3 LC,49,Houston North Wayside,1,100.0,88101,PM2.5 - Local Conditions,209.0,Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,201,Harris,29.828086,-95.284096,2021-05-08,2021
734,05/09/2021,AQS,482010046,1,10.5,ug/m3 LC,54,Houston North Wayside,1,100.0,88101,PM2.5 - Local Conditions,209.0,Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,201,Harris,29.828086,-95.284096,2021-05-09,2021
735,05/10/2021,AQS,482010046,1,13.0,ug/m3 LC,58,Houston North Wayside,1,100.0,88101,PM2.5 - Local Conditions,209.0,Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,201,Harris,29.828086,-95.284096,2021-05-10,2021
736,05/11/2021,AQS,482010046,1,11.0,ug/m3 LC,55,Houston North Wayside,1,100.0,88101,PM2.5 - Local Conditions,209.0,Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,201,Harris,29.828086,-95.284096,2021-05-11,2021
737,05/12/2021,AQS,482010046,1,6.0,ug/m3 LC,33,Houston North Wayside,1,100.0,88101,PM2.5 - Local Conditions,209.0,Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,201,Harris,29.828086,-95.284096,2021-05-12,2021
738,05/13/2021,AQS,482010046,1,9.9,ug/m3 LC,52,Houston North Wayside,1,100.0,88101,PM2.5 - Local Conditions,209.0,Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,201,Harris,29.828086,-95.284096,2021-05-13,2021
739,05/14/2021,AQS,482010046,1,12.1,ug/m3 LC,57,Houston North Wayside,1,100.0,88101,PM2.5 - Local Conditions,209.0,Met One BAM-1022 Mass Monitor w/ VSCC or TE-PM...,26420,"Houston-The Woodlands-Sugar Land, TX",48,Texas,201,Harris,29.828086,-95.284096,2021-05-14,2021
